In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 157.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.1/774.1 kB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 100.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 106.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 149.

In [2]:
!pip install pandas

In [3]:
import pandas as pd

In [84]:
data = pd.read_csv('sample_review_helpful.csv', sep = ";")
data['review'] = data['review'].fillna('')

In [85]:
data['review'] = data['review'].str.lower()

In [5]:
data['helpful']

0         4.0
1         0.0
2         0.0
3         6.0
4        15.0
         ... 
40003     0.0
40004     9.0
40005     0.0
40006     0.0
40007     0.0
Name: helpful, Length: 40008, dtype: float64

In [6]:
(data['helpful'] != 0).astype(int)

0        1
1        0
2        0
3        1
4        1
        ..
40003    0
40004    1
40005    0
40006    0
40007    0
Name: helpful, Length: 40008, dtype: int64

In [86]:
data = pd.DataFrame({'text': data['review'], 'labels': (data['helpful'] != 0).astype(int)})

In [10]:
!pip install scikit-learn

In [87]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(data, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=42)

In [151]:
from simpletransformers.classification import ClassificationModel
from simpletransformers.classification import ClassificationArgs
model_args = ClassificationArgs(
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    max_seq_length=128
)
# Create a TransformerModel
#model = ClassificationModel('roberta', 'roberta-base')
model = ClassificationModel('roberta', 'roberta-large', args=model_args, use_cuda = True)
#model = ClassificationModel('bert', 'bert-base-uncased', args=model_args, use_cuda=True)
#model = ClassificationModel('roberta', 'roberta-base', args=model_args, use_cuda=True)
#model = ClassificationModel('bert', 'bert-large-uncased', args=model_args, use_cuda=True)
#model = ClassificationModel('distilbert', 'distilbert-base-uncased', args=model_args, use_cuda=True)
#model = ClassificationModel('albert', 'albert-base-v2', args=model_args, use_cuda=True)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import os
#os.environ["OMP_NUM_THREADS"] = "1"

In [13]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [152]:
history = model.train_model(train_df, eval_data=val_df, args={'num_train_epochs': 2, 'learning_rate': 3e-5, 'overwrite_output_dir': True,  'train_batch_size' : 128})

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/188 [00:00<?, ?it/s]

Running Epoch 2 of 2:   0%|          | 0/188 [00:00<?, ?it/s]

In [16]:
history

(940, 0.3177600383600022)

In [154]:
result, model_outputs, wrong_predictions = model.eval_model(data)

Running Evaluation:   0%|          | 0/401 [00:00<?, ?it/s]

In [161]:
model_outputs_data = pd.concat([data, pd.DataFrame(model_outputs)], axis = 1)

In [162]:
model_outputs_data.to_csv('model_outputs_data.csv', sep = ";")

In [146]:
result

{'mcc': 0.5556927273033209,
 'accuracy': 0.7776805798550362,
 'f1_score': 0.7736929143874826,
 'tp': 3041,
 'tn': 3182,
 'fp': 820,
 'fn': 959,
 'auroc': 0.8605386994002998,
 'auprc': 0.8596520398701926,
 'eval_loss': 0.464546744838173}

In [124]:
results = pd.DataFrame(result, index = [0])
results['train_loss'] = history[1]
results['name'] = 'roberta roberta-large'
results['seq_length'] = 64

In [148]:
results_new = pd.DataFrame(result, index = [0])
results_new['train_loss'] = history[1]
results_new['name'] = 'roberta roberta-large'
results_new['seq_length'] = 512
results = pd.concat([results, results_new])

In [144]:
results.reset_index(drop = True, inplace = True)

In [149]:
results

,mcc,accuracy,f1_score,tp,tn,fp,fn,auroc,auprc,eval_loss,train_loss,name,seq_length
0,0.552112,0.776056,0.776000,3104,3106,896,896,0.847753,0.831580,0.485895,0.509298,roberta roberta-large,64
1,0.566036,0.782929,0.780099,3081,3184,818,919,0.861318,0.850990,0.465169,0.486931,roberta roberta-large,128
2,0.564950,0.782429,0.784289,3165,3096,906,835,0.864256,0.862096,0.457304,0.483222,roberta roberta-large,256
0,0.555693,0.777681,0.773693,3041,3182,820,959,0.860539,0.859652,0.464547,0.509095,roberta roberta-large,512


In [150]:
results.to_csv('roberta_seq_length.csv', sep = ";")

In [55]:
model.save_model("./roberto")

In [14]:
num_epochs = 5
train_loss_list = []
eval_loss_list = []
for epoch in range(num_epochs):
    train_loss = model.train_model(train_df, args={'num_train_epochs': 1, 'learning_rate': 3e-5,  'train_batch_size' : 128})
    result, model_outputs, wrong_predictions = model.eval_model(train_df)
    print("train: ", epoch, " ", result['accuracy'])
    result_val, model_outputs_val, wrong_predictions_val = model.eval_model(val_df)
    print("val: ", epoch, " ", result_val['accuracy'])
    train_loss_list.append(result)
    eval_loss_list.append(result_val)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/188 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/241 [00:00<?, ?it/s]

train:  0   0.7938260289951675


Running Evaluation:   0%|          | 0/81 [00:00<?, ?it/s]

val:  0   0.7756810797300675


ValueError: Output directory (outputs/) already exists and is not empty. Set overwrite_output_dir: True to automatically overwrite.

In [46]:
train_loss_list

[{'mcc': 0.9913661441615053,
  'accuracy': 0.9957023297896404,
  'f1_score': 0.99538315403477,
  'tp': 22530,
  'tn': 25892,
  'fp': 76,
  'fn': 133,
  'auroc': 0.9995070557379769,
  'auprc': 0.9991754932382618,
  'eval_loss': 0.024634547413794387},
 {'mcc': 0.9912414850078363,
  'accuracy': 0.9956406407435586,
  'f1_score': 0.9953180212014134,
  'tp': 22534,
  'tn': 25885,
  'fp': 83,
  'fn': 129,
  'auroc': 0.9994798371278881,
  'auprc': 0.9991642758943962,
  'eval_loss': 0.03136807641873614},
 {'mcc': 0.9910766724202588,
  'accuracy': 0.9955583886821163,
  'f1_score': 0.9952288390174943,
  'tp': 22528,
  'tn': 25887,
  'fp': 81,
  'fn': 135,
  'auroc': 0.99947345069058,
  'auprc': 0.9992033132070258,
  'eval_loss': 0.03537600909059146}]